In [1]:
import pandas as pd
import numpy as np
import os
import simplejson
import json

In [2]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [3]:
def file_params(df):
    params = {}
    #size
    params['size'] = {}
    params['size']['rows'] = int(df.shape[0])
    params['size']['cols'] = int(df.shape[1])

    #missing values
    params['missing'] = {}
    ##rows
    params['missing']['rows'] = int(df[df.isna().any(axis=1)].shape[0])
    params['missing']['rowsPercent'] = float(round(params['missing']['rows'] / params['size']['rows'], 7) * 100)
    ##cells
    params['missing']['cells'] = int(df.isna().sum().sum())
    params['missing']['cellsPercent'] = float(round(params['missing']['cells'] / (params['size']['rows'] * params['size']['cols']), 7) * 100)
    ##columns
    missingColumn = df.isna().sum()
    params['missing']['cols'] = missingColumn[missingColumn != 0].to_dict()
    ###percent of total values in colums
    nanByColumnPercent = round(df.isna().sum() / df.count().sum() * 100, 4)
    params['missing']['colsPercent'] = nanByColumnPercent[nanByColumnPercent !=0].to_dict()
    ###percent ot total missing values
    nanColumnContributionPercent = round(df.isna().sum() / df.isna().sum().sum() * 100, 2)
    params['missing']['colsPercentContribution'] = nanColumnContributionPercent[nanColumnContributionPercent !=0].to_dict()

    #names
    params['names'] = {}
    params['names']['cols'] = list(df.columns.values)
    params['names']['colsReverse'] = list(df.columns.values)
    params['names']['colsReverse'].reverse()

    #describe
    params['describe'] = df.describe().to_dict()

    return params

In [4]:

target = 'NFLness'

df_sub1 = load_file('/Users/brandon/Desktop/test_data/NFL-Gen-Test_missing.csv')



df_sub2 = load_file('/Users/brandon/Desktop/test_data/NFL-Training_missing.csv')

df = pd.concat([df_sub1, df_sub2])

True

In [11]:
df.shape

(488, 10)

In [7]:
df.iloc[0:5, 0:5]]

SyntaxError: unmatched ']' (3213029144.py, line 1)